# Read and view Delft3D grid, depth and enclosure files

In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
from cmocean.cm import deep
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
from JulesD3D.dep import Dep
from JulesD3D.grid import Grid
from JulesD3D.enc import readEnc
from JulesD3D.bnd import bnd
from os import path, walk
colormap = deep
%load_ext autoreload

In [3]:
%reload_ext autoreload

## Load grid + depth + boundaries

In [4]:
folder_name = "/Users/julesblom/Dropbox/TU/Master/Thesis/Runs/Current/Replenishment_TransportLayer50cm/Run01"

In [5]:
basename = path.basename(folder_name)
plot_title, _ = path.splitext(basename)
title = basename

In [6]:
all_filenames = []

# Find filenames in template/read folder
for root, dirs, files in walk(folder_name):
    for file in files:
        if file.endswith('bnd'):
            bnd_filename = path.join(folder_name, file)
            all_filenames.append({"type": "Boundary", "filename": path.split(bnd_filename)[1]})
        elif file.endswith('dep'):
            dep_filename =  path.join(folder_name, file)
            all_filenames.append({"type": "Depth", "filename": path.split(dep_filename)[1]})
        elif file.endswith('enc'):
            enc_filename =  path.join(folder_name, file)
            all_filenames.append({"type": "Enclosure", "filename":path.split(enc_filename)[1]})
        elif file.endswith('grd'):
            grid_filename = path.join(folder_name, file)
            all_filenames.append({"type": "Grid", "filename":path.split(grid_filename)[1]})        

In [7]:
files_df = pd.DataFrame(data=all_filenames)
files_df

,type,filename
0,Depth,gen_36_by_26km.dep
1,Boundary,QChannelZdeep.bnd
2,Grid,36km_26km_W60Channel.grd
3,Enclosure,36km_1cellchannel_15km.enc


In [8]:
grid = Grid.fromfile(grid_filename)
grid.shape

(181, 131)

In [9]:
dep = Dep.read(dep_filename, grid.shape)
depth = dep.values[0:-1,0:-1]

In [10]:
enclosure_x, enclosure_y = readEnc(enc_filename)

,x,y
0,1,75
1,64,75
2,64,1
3,69,1
4,69,75
5,132,75
6,132,182
7,1,182
8,1,75


In [11]:
boundary_coords = bnd.readBnd(fname=bnd_filename)

,name,type,forcing,m1,n1,m2,n2,reflection coefficient,vertical profile,label1,label2
0,Discharge,Q,T,66,1,66,1,200.0,3d-profile,NaN,NaN
1,deep,Z,T,2,182,131,182,200.0,NaN,NaN,NaN


## Map plot

In [12]:
x_gridcells, y_gridcells = grid.x.shape

In [13]:
# almost!
# xGridstep, yGridstep = [int((grid.x[-1][-1] / grid.y.shape[1])+1), int((grid.y[-1][-1] / grid.y.shape[0])+1)]
# print(xGridstep, yGridstep)

In [14]:
x_gridstep = 200
y_gridstep = 200

In [15]:
fig_length, fig_width = [x_gridcells/10, y_gridcells/10]
min_depth, max_depth = [np.amin(depth), np.max(depth)]

In [16]:
# Minus 1 cause off by 1
plot_enclosure_x_coords = [i - 1 for i in enclosure_x] # oo GIFs afac
plot_enclosure_y_coords = [i - 3 for i in enclosure_y]

In [17]:
plot_enclosure_x_meters = [i * x_gridstep for i in plot_enclosure_x_coords]
plot_enclosure_y_meters = [i * y_gridstep for i in enclosure_y]

In [18]:
bc_x_coords, bc_y_coords = bnd.getXY(boundary_coords)

In [19]:
bc_x_meters = [(i[0] * x_gridstep, i[1] * y_gridstep) for i in bc_x_coords]
bc_y_meters = [(i[0] * x_gridstep, i[1] * y_gridstep) for i in bc_y_coords]

In [20]:
deep_1 = (bc_x_meters[1][0], bc_y_meters[1][0])
deep_2 = (bc_x_meters[1][1], bc_y_meters[1][1])
deep_bc_meters = (deep_1, deep_2)
deep_bc_meters

((400, 36400), (26200, 36400))

In [21]:
discharge_location_x = bc_x_meters[0][0]
discharge_location_y = bc_y_meters[0][0]
(discharge_location_x, discharge_location_y)

(13200, 200)

In [22]:
x_grid = grid.x.data[0]
y_grid = grid.y.data[:,1]

In [23]:
# tickstep = 10
# coords_x_ticks = [i for i in range(0, x_gridcells, tickstep)]
# coords_y_ticks = [i for i in range(0, y_gridcells, tickstep)]

In [27]:
fig, [gridAx0, gridAx1] = plt.subplots(1,2, figsize=(10, 5))

fig.suptitle(title, fontsize=13)
# In M by N numbers
gridAx0.set_title('In grid numbers')
gridAx0.set_aspect('equal')
gridAx0.plot(plot_enclosure_x_coords, plot_enclosure_y_coords, c='#ffb9be', linewidth=3)
gridAx0.pcolor(depth, vmin=min_depth, vmax=max_depth, cmap=colormap) 
gridAx0.set_xlabel('m')
gridAx0.set_ylabel('n')
# gridAx0.set_xticks(coords_x_ticks)
# gridAx0.set_xticklabels(labels=coords_x_ticks, rotation='vertical')
gridAx0.grid(linewidth=0.5, alpha=0.8, color='w')# xdata=newGrid.x[0][1:-1], ydata=newGrid.y[0][1:-1])
gridAx0.scatter(bc_x_coords, bc_y_coords, c='black', linewidth=3)

# In meters

gridAx1.set_title('In meters')
gridAx1.set_aspect('equal')
gridAx1.plot(plot_enclosure_x_meters, plot_enclosure_y_meters, c='#ffb9be', linewidth=1.5)
grid_im = gridAx1.pcolor(grid.x, grid.y, depth[0:-1,0:-1], vmin=min_depth, vmax=max_depth, cmap=deep)
gridAx1.set_ylabel('Length $n$ [m]', fontsize=10)
gridAx1.set_ylabel('Length $n$ [m]', fontsize=10)
gridAx1.set_xlim((0, 26200))
gridAx1.set_ylim(0, 36700)
# gridAx1.set_xticks([i for i in range(0,19000,2500)])
# gridAx1.set_xticklabels(labels=[i for i in range(0,17000,1000)], rotation='vertical')
gridAx1.grid(linewidth=0.5, alpha=0.7, color='w')# xdata=newGrid.x[0][1:-1], ydata=newGrid.y[0][1:-1])
discharge_point = gridAx1.scatter(discharge_location_x, discharge_location_y, c='black', marker="^", label="Discharge", linewidths=7, edgecolors="white")
fig.colorbar(grid_im, ax=gridAx1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Report plot

In [29]:
fig_r, ax_r = plt.subplots(nrows=1, figsize=(7, 8))

# axins = zoomed_inset_axes(ax_r, 2.5, loc=2) # zoom-factor: 2.5, location: upper-left

ax_r.set_title('Map view of model domain')
ax_r.set_aspect('equal')
ax_r.plot(bc_x_meters[1:][0], bc_y_meters[1:][0], c='coral', label='Zero discharge BC', linewidth=7.5)
ax_r.plot(plot_enclosure_x_meters, plot_enclosure_y_meters, c='white', linewidth=2.5, label="Enclosure")
grid_im = ax_r.pcolor(grid.x, grid.y, depth[0:-1,0:-1], vmin=min_depth, vmax=max_depth, cmap=deep)
ax_r.set_xlabel('Width $m$ [m]', fontsize=16)
ax_r.set_ylabel('Length $n$ [m]', fontsize=16)
ax_r.set_xlim(0, 26200)
ax_r.set_ylim(0, 36700)

# grid
ax_r.set_xticks(x_grid, minor=True)
ax_r.set_yticks(y_grid, minor=True)

ax_r.grid(which='minor', alpha=0.15)
ax_r.grid(which='major', alpha=0.75)

discharge = ax_r.scatter(discharge_location_x - 100, discharge_location_y + 400, s=[175], c='coral', label="Discharge BC", marker="^", edgecolors="white")  #

# colorbar
cbar = fig_r.colorbar(grid_im, ax=ax_r)
cbar.ax.set_ylabel("Depth [m]", rotation=-90, va="bottom", fontsize=16)

# legend 
fig_r.legend(loc=(0.15,.273))
# fig_r.legend(fontsize="xx-large", loc="lower left")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 3D depth

In [ ]:
plt.close("all")

In [ ]:
fig_3d = plt.figure(figsize=(15,10))
ax_3d = fig_3d.gca(projection='3d')

print(depth.shape)
print(grid.x.shape)

surface = ax_3d.plot_surface(grid.x, grid.y, depth, cmap=colormap, label='Initial Bathymetry')
fig_3d.colorbar(surface, shrink=0.5, aspect=5, label='Depth [m]')

ax_3d.set_title("3D depth " + title, fontsize=13)
ax_3d.set_xlabel('X [m]')
ax_3d.set_ylabel('Y [m]')
ax_3d.set_zlabel('Depth [m]')
ax_3d.view_init(45, 70)

ax_3d.set_zlim(max_depth, min_depth)

# PyVista 3D Depth Widget

In [ ]:
import pyvista as pv
from cmocean.cm import deep_r
# from JulesD3D.plotPyVista import makeBottomSurface

In [ ]:
grid.x[:-1,:-1].shape

In [ ]:
plot_x_mesh = grid.x[:-1,:-1]
plot_y_mesh = grid.y[:-1,:-1]
plot_z_mesh = -depth[:-1,:-1]

bottom_surface = pv.StructuredGrid(plot_x_mesh, plot_y_mesh, plot_z_mesh)
bottom_surface.field_arrays['depth'] = -depth[:-1,:-1].T
bottom_surface

In [ ]:
p = pv.Plotter(notebook=True)
p.add_mesh(bottom_surface, cmap=deep_r, scalars='depth', ambient=0.2)

p.show_grid()
p.set_scale(zscale=25)
p.show(screenshot="rememberbathymetry.png")

## Plot grid only

In [ ]:
# Flipped y and x for nice plot!
gridFig, gridAx = plt.subplots(figsize=(20, 10))
gridAx.plot(grid.y.transpose(), grid.x.transpose(), 'gray')
gridAx.plot(grid.y, grid.x, 'gray')
gridAx.set_xlabel('n [m]')
gridAx.set_ylabel('m [m]')
gridAx.set_aspect('equal')

In [ ]:
plt.close('all')